In [57]:
import tensorflow as tf
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn as sklearn
from sklearn import preprocessing
import os

In [4]:
# 读取数据
testdata = pd.read_csv('../data/dataannalysis/happiness_test_abbr.csv')
traindata = pd.read_csv('../data/dataannalysis/happiness_train_abbr.csv')

In [5]:
#异常数据处理
traindata = traindata[traindata['happiness'] != -8]
traindata = traindata[traindata['depression'] != -8]
traindata = traindata[traindata['class'] != -8]
traindata = traindata[traindata['health'] != -8]
traindata = traindata[traindata['equity'] != -8]
traindata = traindata[traindata['family_status'] != -8]
traindata = traindata[traindata['health_problem'] != -8]
traindata = traindata[traindata['relax'] != -8]
traindata = traindata[traindata['learn'] != -8]
traindata['view'].replace(-8,3,inplace = True)
traindata = traindata[traindata['socialize'] != -8]
traindata = traindata[traindata['socialize'] != -1]
traindata = traindata[traindata['socialize'] != -2]
traindata = traindata[traindata['socialize'] != -3]
traindata = traindata[traindata['socialize'] != 50]
traindata = traindata[traindata['status_3_before'] != -8]
traindata = traindata[traindata['status_peer'] != -8]
traindata['inc_ability'].replace(-8,3,inplace = True)

#数据重新处理
traindata['marital'].replace(2,1,inplace = True)
traindata['marital'].replace(3,1,inplace = True)
traindata['marital'].replace(4,1,inplace = True)
traindata['marital'].replace(7,1,inplace = True)
traindata['marital'].replace(6,2,inplace = True)
traindata['marital'].replace(5,3,inplace = True)

bins = [-100,0,100000,300000,500000,1000000,10000000] 
traindata['income'] = pd.cut(traindata['income'],bins, labels=['1','2', '3', '4', '5', '6'])



avg_income=traindata['family_income']/traindata['family_m']
traindata['avg_income'] = avg_income
bins = [-100,0,20000,50000,100000,1000000,10000000] 
traindata['avg_income'] = pd.cut(traindata['avg_income'],bins, labels=['1','2', '3', '4', '5', '6'])

In [6]:
#traindata['avg_income'].('unknown', inplace=True)
traindata['avg_income'] = traindata['avg_income'].cat.add_categories(['0']);
traindata['avg_income'].fillna('0', inplace=True)

le = sklearn.preprocessing.LabelEncoder() 
le.fit(['1','2', '3', '4', '5', '6', '0']) 
traindata['avg_income']  = le.transform(traindata['avg_income'] ) 

le = sklearn.preprocessing.LabelEncoder() 
le.fit(['1','2', '3', '4', '5', '6']) 
traindata['income']  = le.transform(traindata['income'] ) 

In [7]:
traindatafinal = traindata[['happiness','depression','class','health','equity','family_status','health_problem','relax','learn','view','socialize','status_peer','status_3_before','inc_ability','car','avg_income','edu']]

In [8]:
train_data = traindatafinal[:7000]
test_data = traindatafinal[7000:]
 
train_data_X = train_data.drop(['happiness'],axis=1)
train_data_Y = train_data['happiness']
test_data_X = test_data.drop(['happiness'],axis=1)
test_data_Y = test_data['happiness']

In [23]:
print(train_data_X.shape)
print(train_data_Y.shape)
print(test_data_X.shape)
print(test_data_Y.shape)
#train_data_X_ =train_data_X.T
#test_data_X_ = test_data_X.T
#print(train_data_X_.shape)
#print(test_data_X_.shape)
#train_data_Y_ = train_data_Y.reshape((1, train_data_Y.shape[0]))
#test_data_tY_ = test_data_Y.reshape((1, test_data_Y.shape[0]))
test_data_Y_ = pd.get_dummies(test_data_Y)
#train_data_Y_ = pd.get_dummies(train_data_Y).T #get_dummies one hot
train_data_Y_ = pd.get_dummies(train_data_Y)

print(train_data_Y_.shape)
print(test_data_Y_.shape)

(7000, 16)
(7000,)
(718, 16)
(718,)
(7000, 5)
(718, 5)


In [36]:
train_data_X =train_data_X.astype(np.float32)
x1=train_data_X-np.mean(train_data_X,axis=0)
normal_train_data_X=x1/np.std(x1,axis=0)
##normal_train_data_X=tf.nn.local_response_normalization(train_data_X,1,0,1,1) 
normal_train_data_X.head()

,depression,class,health,equity,family_status,health_problem,relax,learn,view,socialize,status_peer,status_3_before,inc_ability,car,avg_income,edu
0,1.258736,-0.816038,-0.583207,-0.193497,-0.928463,-1.759826,0.730827,0.973896,0.564666,-0.773327,1.302760,0.388394,1.026235,0.397537,0.783352,1.948324
1,-0.919008,1.032991,1.298972,-0.193497,1.875956,0.131443,0.730827,0.973896,0.564666,-0.773327,-2.349351,-1.227028,-0.722462,0.397537,-0.427660,2.267199
2,1.258736,0.416648,0.357883,0.804647,0.473747,0.131443,0.730827,0.047258,0.564666,0.186816,-0.523295,-1.227028,-0.722462,0.397537,-0.427660,-0.283798
3,0.169864,0.416648,0.357883,0.804647,0.473747,0.131443,0.730827,1.900533,-0.827116,-0.773327,-0.523295,-1.227028,-0.722462,-1.857533,-0.427660,-0.602673
4,-0.919008,-2.048723,1.298972,-1.191642,0.473747,1.077078,-0.331642,1.900533,-0.827116,1.146960,1.302760,0.388394,1.026235,-1.857533,-1.638673,2.267199


In [43]:

#创建一个神经网络层
def add_layer(input,in_size,out_size,activation_function=None):
    """
    :param input: 数据输入
    :param in_size: 输入大小
    :param out_size: 输出大小
    :param activation_function: 激活函数（默认没有）
    :return:output：数据输出
    """
    Weight=tf.Variable(tf.random_normal([in_size,out_size]) )
    biases=tf.Variable(tf.zeros([1,out_size]) +0.1 )
    W_mul_x_plus_b=tf.matmul(input,Weight) + biases
    #根据是否有激活函数
    if activation_function == None:
        output=W_mul_x_plus_b
    else:
        output=activation_function(W_mul_x_plus_b)
    return output
lambd

0.01

In [44]:
lambd =0.01
regularizer = tf.contrib.layers.l2_regularizer(lambd)
#定义输入数据
xs=tf.placeholder(tf.float32,[None,16])
ys=tf.placeholder(tf.float32,[None,5])
 
#定义一个隐藏层
hidden_layer1=add_layer(xs,16,10,activation_function=tf.nn.relu)
hidden_layer2=add_layer(hidden_layer1,10,7,activation_function=tf.nn.relu)
#定义一个隐藏层
#hidden_layer1=add_layer(xs,16,10,activation_function=tf.nn.relu)
#定义一个输出层
prediction=add_layer(hidden_layer2,7,5,activation_function=None)
 
#求解神经网络参数
#1.定义损失函数
loss=tf.reduce_mean(tf.reduce_sum(tf.square(ys-prediction) ,reduction_indices=[1] ))
#2.定义训练过程
train_step=tf.train.GradientDescentOptimizer(0.1).minimize(loss)
 
init=tf.global_variables_initializer()
sess=tf.Session()
sess.run(init)
#3.进行训练
for i in range(1000):
    sess.run(train_step,feed_dict={xs:train_data_X,ys:train_data_Y_})
    if i%100==0:
        print(sess.run(loss,feed_dict={xs:train_data_X,ys:train_data_Y_} )  )
 
#关闭sess
sess.close()


2027020800000000.0
1188.3124
0.5760737
0.5760737
0.5760737
0.5760737
0.5760737
0.5760737
0.5760737
0.5760737


In [62]:
def fc_layers(input_tensor,regularizer):
    HINDENN1 = 7
    HINDENN2 = 5
    with tf.name_scope("full-connect-layer"):
        fc1 = tf.layers.dense(input_tensor, HINDENN1, activation=tf.nn.elu,\
            kernel_regularizer=regularizer)
        fc2 = tf.layers.dense(fc1, HINDENN2, activation=tf.nn.elu,\
            kernel_regularizer=regularizer)
    return fc2

In [71]:
def random_batch(X_train, y_train, batch_index, batch_size):
    
    #rnd_indices = np.random.randint(0, len(X_train), batch_size)
    #X_batch = X_train[rnd_indices]
   # y_batch = y_train[rnd_indices]arr7[1:3,:]
    X_batch = X_train[batch_index*batch_size:(batch_index+1)*batch_size,:]
    y_batch = y_train[batch_index*batch_size:(batch_index+1)*batch_size,:]
    return X_batch, y_batch

In [73]:
n_inputs = train_data_X_.shape[1]
n_outputs = len(set(train_data_Y))
learning_rate=0.01
with tf.name_scope("input"):
    X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
    y = tf.placeholder(tf.int32, shape=(None), name="y")

regularizer = tf.contrib.layers.l2_regularizer(lambd)
fc2 = fc_layers(X,regularizer)
with tf.name_scope("output"):
    logits = tf.layers.dense(fc2, n_outputs, kernel_regularizer=regularizer,name="output3")

with tf.name_scope('loss'):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y, logits= logits)
    loss = tf.reduce_mean(xentropy, name = 'loss')
    loss_summary = tf.summary.scalar('loss', loss)

global_step = tf.Variable(0, trainable = False)
with tf.name_scope('train'):
    optimizer = tf.train.AdamOptimizer(learning_rate)
    grads_and_vars = optimizer.compute_gradients(loss)
    train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

with tf.name_scope('eval'):
    predictions = tf.argmax(logits, 1)
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    acc_summary = tf.summary.scalar('acc', accuracy)

summary_op = tf.summary.merge([loss_summary, acc_summary])

In [74]:
checkpoint_path = "./chickpoints/model.ckpt"
checkpoint_epoch_path = checkpoint_path + ".epoch"
final_model_path = "./chickpoints/model"

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
logdir = './logs/'+ now
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())
saver = tf.train.Saver()

NameError: name 'datetime' is not defined

In [75]:
n_epochs = 2000
batch_size = 50
n_batches = int(np.ceil(train_data_X.shape[0] / batch_size))
with tf.Session() as sess:
    init = tf.global_variables_initializer()

    if os.path.isfile(checkpoint_epoch_path):
        # if the checkpoint file exists, restore the model and load the epoch number
        with open(checkpoint_epoch_path, "rb") as f:
            start_epoch = int(f.read())
        print("Training was interrupted. Continuing at epoch", start_epoch)
        saver.restore(sess, checkpoint_path)
    else:
        start_epoch = 0
        sess.run(init)

    for epoch in range(start_epoch, n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = random_batch(normal_train_data_X, train_data_Y,batch_index, batch_size)
            sess.run(train_op, feed_dict={X: X_batch, y: y_batch})
        loss_val, summary_str,test_pred, test_acc = sess.run(
                                        [loss, summary_op,predictions, accuracy],\
                                        feed_dict={X: X_test, y: y_test})

        file_writer.add_summary(summary_str, epoch)
        if epoch % 50 == 0:
            print("Epoch:", epoch, "\tLoss:", loss_val,"\tAcc:",test_acc)
            saver.save(sess, checkpoint_path)
            with open(checkpoint_epoch_path, "wb") as f:
                f.write(b"%d" % (epoch + 1))

    saver.save(sess, final_model_path)
    y_pred = predictions.eval(feed_dict={X: X_test, y: y_test})
    print('precision_score',precision_score(y_test, y_pred))
    print('recall_score',recall_score(y_test, y_pred))

    sess.close()

TypeError: unhashable type: 'slice'

In [68]:
arr7 = np.arange(35).reshape(5,7)#生成一个5*7的数组

arr7



array([[ 0,  1,  2,  3,  4,  5,  6],
       [ 7,  8,  9, 10, 11, 12, 13],
       [14, 15, 16, 17, 18, 19, 20],
       [21, 22, 23, 24, 25, 26, 27],
       [28, 29, 30, 31, 32, 33, 34]])

In [70]:

arr7[1:3,:]

array([[ 7,  8,  9, 10, 11, 12, 13],
       [14, 15, 16, 17, 18, 19, 20]])